In [39]:
from typing import List, Dict, Union

import simpy
import pandas as pd
import pybamm
import numpy as np

pybamm.set_logging_level("INFO")

In [40]:
class PybammBattery:

    def __init__(self, env, capacity, soc=0, Vmin=2.8, Vmax=4.5):
        self.env = env
        self.capacity = capacity
        self.soc = soc
        self.excess_power = 0
        self.Vmin = Vmin
        self.Vmax = Vmax
        self.c_n_min = 0
        self.c_n_max = 0
        self.c_p_min = 0
        self.c_p_max = 0
        self.step_solution = None
        
        # load solver
        self.solver = pybamm.CasadiSolver()
        
        # load model
        self.model = pybamm.lithium_ion.SPMe()
        
        # load parameter values and process model and geometry
        self.parameter_values = pybamm.ParameterValues('Chen2020')
        
        # load values for c_n_min, c_n_max, c_p_min and c_p_max
        self.calculate_CN_CP_values()
        
        geometry = self.model.default_geometry
        self.parameter_values['Current function [A]'] = "[input]"
        self.parameter_values.process_model(self.model)
        self.parameter_values.process_geometry(geometry)
        
        # set mesh
        mesh = pybamm.Mesh(geometry, self.model.default_submesh_types, self.model.default_var_pts)

        # discretise model
        disc = pybamm.Discretisation(mesh, self.model.default_spatial_methods)
        disc.process_model(self.model)
        
        
    def calculate_CN_CP_values(self):
        esoh_model = pybamm.lithium_ion.ElectrodeSOH()
        esoh_sim = pybamm.Simulation(esoh_model, parameter_values=self.parameter_values)
        param = self.model.param
        
        self.parameter_values['Lower voltage cut-off [V]'] = self.Vmin
        self.parameter_values['Upper voltage cut-off [V]'] = self.Vmax
        
        print(f"Lower voltage cut-off [V]': {self.Vmin:.3f}")
        print(f"Upper voltage cut-off [V]': {self.Vmax:.3f}")
        
        Cn = self.parameter_values.evaluate(param.C_n_init)
        Cp = self.parameter_values.evaluate(param.C_p_init)
        n_Li_init = self.parameter_values.evaluate(param.n_Li_particles_init)
        
        esoh_sol = esoh_sim.solve(
            [0], 
            inputs={"V_min": self.Vmin, "V_max": self.Vmax, "C_n": Cn, "C_p": Cp, "n_Li": n_Li_init}
        )
        print(f"Initial negative electrode SOC: {esoh_sol['x_100'].data[0]:.3f}")
        print(f"Initial positive electrode SOC: {esoh_sol['y_100'].data[0]:.3f}")
        
        self.parameter_values.update(
            {
                "Initial concentration in negative electrode [mol.m-3]": esoh_sol["x_100"].data[0],
                "Initial concentration in positive electrode [mol.m-3]": esoh_sol["y_100"].data[0],
            }
        )
        
        self.c_n_min = esoh_sol["x_0"].data[0]
        self.c_n_max = esoh_sol["x_100"].data[0]
        self.c_p_min = esoh_sol["y_0"].data[0]
        self.c_p_max = esoh_sol["y_100"].data[0]
        
        print(f"Minimum negative particle concentration: {self.c_n_min:.3f}")
        print(f"Maximum negative particle concentration: {self.c_n_max:.3f}")
        print(f"Minimum positive particle concentration: {self.c_p_min:.3f}")
        print(f"Maximum positive particle concentration: {self.c_p_max:.3f}")
        
        
    def update(self, current):
        if current == 0:
            return 0
        
        if self.step_solution is not None:
            print('condition final time: ', not (
            self.step_solution.termination == "final time"
            or "[experiment]" in self.step_solution.termination
        ))
        
        input_parameters= {}
        input_parameters['Current function [A]'] = current
        self.step_solution = self.solver.step(self.step_solution, self.model, dt=120, npts=100, inputs=input_parameters)
        
        self.calculate_soc()
        
        
    def calculate_soc(self):
        c_n_data = self.step_solution['Average negative particle concentration [mol.m-3]'].data
        c_p_data = self.step_solution['Average positive particle concentration [mol.m-3]'].data
        print('negative electrode: ', self.step_solution['Negative electrode SOC'].data[-1])
        print('positive electrode: ', self.step_solution['Positive electrode SOC'].data[-1])
        print('')

        SoC_from_n = (c_n_data - self.c_n_min) / (self.c_n_max - self.c_n_min)
        SoC_from_p = (c_p_data - self.c_p_min) / (self.c_p_max - self.c_p_min)

        print('SoC match', np.allclose(SoC_from_n, SoC_from_p))
        print('SoC_from_n: ', SoC_from_n[-1])
        print('SoC_from_p: ', SoC_from_p[-1])

In [41]:
def simulate(env: simpy.Environment, battery: PybammBattery, current_delta_list: List[float], records: List[Dict]):
    
    for current_delta in current_delta_list:
        yield env.timeout(1)
        battery.update(current_delta)
        records.append({
            "power_delta": current_delta,
            "excess_power": battery.excess_power,
            "soc": battery.soc,
            "capacity [A.h]": battery.capacity
        })

In [42]:
# For now let's assume the simple case of one step every second where we first (dis)charge and then implicitly read.
# Later we can extend this to a more asynchronous charge/discharge/read pattern with different processes if we want
current_delta_list = [-0.52, 0.069, 1, -1, -0.65, 1]
records = []  # log of some infos for later analysis

env = simpy.Environment()
battery = PybammBattery(env, capacity=5)
env.process(simulate(env, battery, current_delta_list, records))
env.run()

result = pd.DataFrame(records)
with open("result.csv", "w") as f:
    f.write(result.to_csv())
print(result)

2022-04-28 16:28:53,034 - [INFO] base_battery_model.build_model(834): Start building Single Particle Model with electrolyte
2022-04-28 16:28:53,095 - [INFO] base_battery_model.build_model(854): Finish building Single Particle Model with electrolyte
2022-04-28 16:28:53,148 - [INFO] parameter_values.process_model(415): Start setting parameters for Electrode-specific SOH model
2022-04-28 16:28:53,312 - [INFO] parameter_values.process_model(518): Finish setting parameters for Electrode-specific SOH model
2022-04-28 16:28:53,314 - [INFO] discretisation.process_model(137): Start discretising Electrode-specific SOH model


Lower voltage cut-off [V]': 2.800
Upper voltage cut-off [V]': 4.500


2022-04-28 16:28:53,359 - [INFO] discretisation.process_model(254): Finish discretising Electrode-specific SOH model
2022-04-28 16:28:53,359 - [INFO] base_solver.solve(815): Start solving Electrode-specific SOH model with Algebraic solver (lm)
2022-04-28 16:28:53,360 - [INFO] base_solver.set_up(111): Start solver set-up
2022-04-28 16:28:53,369 - [INFO] base_solver.set_up(678): Finish solver set-up
2022-04-28 16:28:53,372 - [INFO] base_solver.solve(1088): Finish solving Electrode-specific SOH model (success)
2022-04-28 16:28:53,373 - [INFO] base_solver.solve(1089): Set-up time: 9.460 ms, Solve time: 2.524 ms (of which integration time: 2.002 ms), Total time: 11.983 ms
2022-04-28 16:28:53,379 - [INFO] parameter_values.process_model(415): Start setting parameters for Single Particle Model with electrolyte
2022-04-28 16:28:53,456 - [INFO] parameter_values.process_model(518): Finish setting parameters for Single Particle Model with electrolyte
2022-04-28 16:28:53,458 - [INFO] discretisation

Initial negative electrode SOC: 1.152
Initial positive electrode SOC: 0.103
Minimum negative particle concentration: 0.040
Maximum negative particle concentration: 1.152
Minimum positive particle concentration: 0.845
Maximum positive particle concentration: 0.103


2022-04-28 16:28:53,720 - [INFO] discretisation.process_model(254): Finish discretising Single Particle Model with electrolyte
2022-04-28 16:28:53,722 - [INFO] base_solver.set_up(111): Start solver set-up
2022-04-28 16:28:53,762 - [INFO] base_solver.set_up(678): Finish solver set-up


negative electrode:  0.003009114795383217
positive electrode:  -0.001983335672427702

SoC match False
SoC_from_n:  89.61994520302302
SoC_from_p:  169.78257393207875
condition final time:  False
negative electrode:  0.002614442193819589
positive electrode:  -0.0017199462879647106

SoC match False
SoC_from_n:  77.86076016646804
SoC_from_p:  147.3864246987837
condition final time:  False
negative electrode:  -0.0031054505824648785
positive electrode:  0.00209729116802068

SoC match False
SoC_from_n:  -92.56221137780692
SoC_from_p:  -177.1954482475215
condition final time:  False
negative electrode:  -0.0015087731550329354
positive electrode:  0.0010317297808579758

SoC match False
SoC_from_n:  -44.989551815230655
SoC_from_p:  -86.59015717708377
condition final time:  True
negative electrode:  -0.0015087731550329354
positive electrode:  0.0010317297808579758

SoC match False
SoC_from_n:  -44.989551815230655
SoC_from_p:  -86.59015717708377
condition final time:  True
negative electrode:  -0

In [ ]:
sorted(battery.model.variables.keys())

In [7]:
battery.model.param